In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
#nltk.download('perluniprops')
from nltk import ngrams
from itertools import chain

import pickle

## Fetch even data

In [2]:
with open('X_train_even.pickle', 'rb') as handle:
    X_train_even = pickle.load(handle)
    
# with open('X_test_even.pickle', 'rb') as handle:
#     X_test_even = pickle.load(handle)
    
with open('Y_train_even.pickle', 'rb') as handle:
    Y_train_even = pickle.load(handle)
    
# with open('Y_test_even.pickle', 'rb') as handle:
#     Y_test_even = pickle.load(handle)

print('Total number of even training data points: %d' %len(X_train_even))

Total number of even training data points: 199483


## First model on even data

In [3]:
# Add your import statements here
from keras import Input, Model
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, Embedding
from keras.layers.recurrent import LSTM
from keras.callbacks import TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential

def create_model(X_seq_len, output_n, hidden_dim, vocabulary_size, embedding_dim):

    model = Sequential()
    model.add(Embedding(vocabulary_size, embedding_dim, input_length=X_seq_len))
    model.add(LSTM(hidden_dim))
    model.add(Dense(output_n,activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

Using TensorFlow backend.


#### Create a smaller data set for testing

In [4]:
lesser_number = 10000

small_Y = Y_train_even[0:lesser_number]
small_X = X_train_even[0:lesser_number]
neg_share = len(small_Y[small_Y==1])/lesser_number

print('The share of negative reviews:')
print(neg_share)
print('It is approximately even!')

The share of negative reviews:
0.4924
It is approximately even!


#### Flatten input data for embedding layer

In [5]:
dim1 = len(small_X)
dim2 = len(small_X[0])

small_X_flat = small_X.reshape(dim1,dim2)

### Train!

In [7]:
X_seq_len = len(small_X[0]) #the input data is padded
output_dim = 1
hidden_dim = 10
num_epochs = 5
b_size = 64

vocabulary_size = 100001 #predefined when creating data
embedding_dim = 10

model = create_model(X_seq_len,output_dim,hidden_dim, vocabulary_size, embedding_dim)
name = 'second_RNN_even_data'

# Define a tensorboard callback
tb = TensorBoard(log_dir='./logs/'+name)
    
# Print model summary and train
model.summary()
current_history = model.fit(small_X_flat, small_Y, verbose=1, batch_size=b_size, epochs=num_epochs, callbacks=[tb], validation_split=0.2);
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 10)           1000010   
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,000,861
Trainable params: 1,000,861
Non-trainable params: 0
_________________________________________________________________
Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 213s 27ms/step - loss: 0.6932 - acc: 0.5045 - val_loss: 0.6931 - val_acc: 0.5045
Epoch 2/5
8000/8000 [==============================] - 199s 25ms/step - loss: 0.6931 - acc: 0.5084 - val_loss: 0.6931 - val_acc: 0.5045
Epoch 3/5
 320/8000 [>.............................] - ETA: 3:07 - loss: 0.6

KeyboardInterrupt: 